In [1]:
import pandas as pd
import numpy as np

import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
start_date = '20221130'
end_date = '20230509'

In [3]:
try:
    tidy_df_30Min = pd.read_parquet(f"../sos/tidy_df_30Min_{start_date}_{end_date}_noplanar_fit.parquet")
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])

In [4]:
def scatter_chart(height = '3', box_width = 200, hist_width = 40, type='both'):
    wide_df = tidy_df_30Min[tidy_df_30Min['variable'].isin([f'Ri_{height}m_c', f'RiB_{height}m_c', f'w_h2o__{height}m_c', f'w_tc__{height}m_c'])].reset_index(drop=True).pivot(
        values = ['value'],
        columns = 'variable',
        index = 'time'
    )
    wide_df.columns = wide_df.columns.get_level_values(1)
    wide_df = wide_df.reset_index()

    rule = alt.Chart().mark_rule().encode(x=alt.datum(0.25))
    xscale = alt.Scale(domain = [-2, 4], clamp=True)
    yscale = alt.Scale(domain = [-.01, .01], clamp=True)

    base = alt.Chart(wide_df)

    ri_points = base.mark_circle(size=2).encode(
        alt.X(f'Ri_{height}m_c').scale(xscale),
        alt.Y(f'w_h2o__{height}m_c').scale(yscale)
    ).properties(width=box_width, height=box_width)

    ri_top_hist = base.mark_bar().encode(
        alt.X(f"Ri_{height}m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)

    ).properties(width=box_width, height=hist_width)

    rule = alt.Chart().mark_rule().encode(x=alt.datum(0.2))
    xscale = alt.Scale(domain = [-2, 4], clamp=True)
    yscale = alt.Scale(domain = [-.01, .01], clamp=True)

    base = alt.Chart(wide_df)

    rib_points = base.mark_circle(size=2).encode(
        alt.X(f'RiB_{height}m_c').scale(xscale),
        alt.Y(f'w_h2o__{height}m_c').scale(yscale)
    ).properties(width=box_width, height=box_width)

    rib_top_hist = base.mark_bar().encode(
        alt.X(f"RiB_{height}m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)
        
    ).properties(width=box_width, height=hist_width)

    w_h2o_right_hist = base.mark_bar().encode(
        alt.Y(f"w_h2o__{height}m_c")
        .bin(maxbins=hist_width, extent=yscale.domain)
        .title(None)
        .axis(labels=False),
        alt.X('count()').title(None)
    ).properties(width=hist_width, height = box_width)

    if type == 'both':
        ri_chart = ri_top_hist & (ri_points + rule)
        rib_chart = rib_top_hist & (rib_points + rule | w_h2o_right_hist)
        return (ri_chart | rib_chart)
        
    elif type == 'ri':
        return ri_top_hist & (ri_points + rule | w_h2o_right_hist)
    
    elif type == 'rib':
        return rib_top_hist & (rib_points + rule | w_h2o_right_hist)
    
    else: 
        print('you get nothing!')
        return None

In [5]:
def heatmap_chart(height = '3', box_width = 200, hist_width = 40, type='both'):
    wide_df = tidy_df_30Min[tidy_df_30Min['variable'].isin([f'Ri_{height}m_c', f'RiB_{height}m_c', f'w_h2o__{height}m_c', f'w_tc__{height}m_c'])].reset_index(drop=True).pivot(
        values = ['value'],
        columns = 'variable',
        index = 'time'
    )
    wide_df.columns = wide_df.columns.get_level_values(1)
    wide_df = wide_df.reset_index()

    rule = alt.Chart().mark_rule().encode(x=alt.datum(0.25))
    # xscale = alt.Scale(domain = [-2, 4], clamp=True)
    # yscale = alt.Scale(domain = [-.01, .01], clamp=True)
    xscale = alt.Scale()
    yscale = alt.Scale()

    # wide_df = wide_df.query(f'Ri_{height}m_c < 4').query(f'Ri_{height}m_c > -2')
    # wide_df = wide_df.query(f'RiB_{height}m_c < 4').query(f'RiB_{height}m_c > -2')

    # wide_df = wide_df.query(f'w_h2o__{height}m_c < 0.01').query(f'w_h2o__{height}m_c > -0.01')

    base = alt.Chart(wide_df)

    ri_points = base.mark_square().encode(
        alt.X(f'Ri_{height}m_c').bin(True).scale(xscale),
        alt.Y(f'w_h2o__{height}m_c').bin(True).scale(yscale),
        alt.Color('count():Q')
    ).properties(width=box_width, height=box_width)

    ri_top_hist = base.mark_bar().encode(
        alt.X(f"Ri_{height}m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)

    ).properties(width=box_width, height=hist_width)

    rule = alt.Chart().mark_rule().encode(x=alt.datum(0.2))
    xscale = alt.Scale(domain = [-2, 4], clamp=True)
    yscale = alt.Scale(domain = [-.01, .01], clamp=True)

    base = alt.Chart(wide_df)

    rib_points = base.mark_square().encode(
        alt.X(f'RiB_{height}m_c').bin(True).scale(xscale),
        alt.Y(f'w_h2o__{height}m_c').bin(True).scale(yscale),
        alt.Color('count():Q')
    ).properties(width=box_width, height=box_width)

    rib_top_hist = base.mark_bar().encode(
        alt.X(f"RiB_{height}m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)
        
    ).properties(width=box_width, height=hist_width)

    w_h2o_right_hist = base.mark_bar().encode(
        alt.Y(f"w_h2o__{height}m_c")
        .bin(maxbins=hist_width, extent=yscale.domain)
        .title(None)
        .axis(labels=False),
        alt.X('count()').title(None)
    ).properties(width=hist_width, height = box_width)

    if type == 'both':
        ri_chart = ri_top_hist & (ri_points + rule)
        rib_chart = rib_top_hist & (rib_points + rule | w_h2o_right_hist)
        return (ri_chart | rib_chart)
        
    elif type == 'ri':
        return ri_top_hist & (ri_points + rule | w_h2o_right_hist)
    
    elif type == 'rib':
        return rib_top_hist & (rib_points + rule | w_h2o_right_hist)
    
    else: 
        print('you get nothing!')
        return None

In [6]:
(
    heatmap_chart(3, type = 'ri')
     | heatmap_chart(5, type = 'ri')
     | heatmap_chart(10, type = 'ri')
     | heatmap_chart(15, type = 'ri')
     | heatmap_chart(20, type = 'ri')
) & (
    heatmap_chart(3, type = 'rib')
     | heatmap_chart(5, type = 'rib')
     | heatmap_chart(10, type = 'rib')
     | heatmap_chart(15, type = 'rib')
     | heatmap_chart(20, type = 'rib')
)

alt.VConcatChart(...)

In [7]:
(
    scatter_chart(3, type = 'ri')
     | scatter_chart(5, type = 'ri')
     | scatter_chart(10, type = 'ri')
     | scatter_chart(15, type = 'ri')
     | scatter_chart(20, type = 'ri')
) & (
    scatter_chart(3, type = 'rib')
     | scatter_chart(5, type = 'rib')
     | scatter_chart(10, type = 'rib')
     | scatter_chart(15, type = 'rib')
     | scatter_chart(20, type = 'rib')
)

alt.VConcatChart(...)